In [1]:
!pip install --upgrade --quiet pip
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.9/187.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00


In [2]:
! pip -q install dspy-ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.4/220.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
from llama_cpp import Llama
# Loading model,
llm = Llama.from_pretrained(
  repo_id="NousResearch/Meta-Llama-3-8B-Instruct-GGUF",
  filename="Meta-Llama-3-8B-Instruct-Q5_K_M.gguf",
  n_ctx=2048,
  n_gpu_layers=-1,
  flash_attn=True,
  verbose=False,
)

Meta-Llama-3-8B-Instruct-Q5_K_M.gguf:   0%|          | 0.00/5.73G [00:00<?, ?B/s]

In [4]:
from dsp.modules.lm import LM
from typing import Any, Literal

try:
    from llama_cpp import Llama
except ImportError:
    raise ModuleNotFoundError("You need to install the llama_cpp library to use gguf models.")

class LlamaCpp(LM):
    def __init__(
            self,
            model: str,
            llama_model: Llama,
            model_type: Literal["chat", "text"] = None,
            **kwargs,
    ):
        super().__init__(model)

        default_model_type = "text"
        self.model_type = model_type if model_type else default_model_type
        self.provider = "llama"

        self.kwargs = {
            "temperature": 0.0,
            "max_tokens": 150,
            "top_p": 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
            **kwargs,
        }  # TODO: add kwargs above for </s>

        self.loaded_model = llama_model
        self.history: list[dict[str, Any]] = []

    def basic_request(self, prompt: str, **kwargs):
        raw_kwargs = kwargs

        kwargs = {**self.kwargs, **kwargs}
        if "n" in kwargs:
            del kwargs["n"]

        if self.model_type == "chat":
            kwargs["messages"] = [{"role": "user", "content": prompt}]
            response = self.loaded_model.create_chat_completion(**kwargs)

        else:
            kwargs["prompt"] = prompt
            response = self.loaded_model.create_completion(**kwargs)

        history = {
            "prompt": prompt,
            "response": response,
            "kwargs": kwargs,
            "raw_kwargs": raw_kwargs,
        }
        self.history.append(history)
        return response

    def request(self, prompt: str, **kwargs):
        if "model_type" in kwargs:
            del kwargs["model_type"]

        return self.basic_request(prompt, **kwargs)

    def _get_choice_text(self, choice) -> str:
        if self.model_type == "chat":
            return choice["message"]["content"]
        return choice["text"]

    def __call__(
            self,
            prompt: str,
            only_completed: bool = True,
            return_sorted: bool = False,
            **kwargs,
    ) -> list[dict[str, Any]]:
        assert only_completed, "for now"
        assert return_sorted is False, "for now"

        response = self.request(prompt, **kwargs)
        choices = response["choices"]

        completed_choices = [c for c in choices if c["finish_reason"] != "length"]

        if only_completed and len(completed_choices):
            choices = completed_choices

        completions = [self._get_choice_text(c) for c in choices]

        return completions

In [5]:
import dspy
llama_cpp_lm = LlamaCpp(model="llama", llama_model=llm)

# Example usage
response = llama_cpp_lm(prompt="Hello, how are you?")
print(response)

[' I hope this message finds you well. My name is [Your Name], and I am reaching out to you today because I am interested in learning more about your company and the services you offer.\n\nI came across your website while researching companies that specialize in [industry/field], and I was impressed by the work you do. Your mission to [briefly describe their mission] resonates with me, and I believe that our values align.\n\nAs someone who is passionate about [related passion], I am excited about the possibility of working together to achieve common goals. I would love to learn more about your team and how we can collaborate to make a positive impact in the industry.\n\nWould you be available for a brief call to discuss further? I am confident']


In [6]:
llama_cpp_lm(prompt="hello")

[", I am a new member here. I have been interested in the topic of quantum computing for some time now and I would like to learn more about it.\nI have a background in computer science and mathematics, but I don't have any formal training in physics or engineering.\n\nI was wondering if there are any resources that you could recommend for someone with my background to learn more about quantum computing?\n\nAlso, I am interested in learning more about the applications of quantum computing, such as cryptography and optimization problems. Are there any specific resources that you would recommend for these topics?\n\nThank you for your time and I look forward to hearing back from you.\n\nBest regards,\n[Your Name]\n\nHello! Welcome to the Quantum Computing community!\n\nI'm happy to help"]

In [7]:
import dspy
dspy.settings.configure(lm=llama_cpp_lm)

In [8]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
# Load math questions from the GSM8K dataset
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

100%|██████████| 1319/1319 [00:00<00:00, 33983.99it/s]


In [9]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)

In [10]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 4-shot examples of our CoT program.
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

# Optimize! Use the `gsm8k_metric` here. In general, the metric is going to tell the optimizer how well it's doing.
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset)

 60%|██████    | 6/10 [00:37<00:25,  6.30s/it]


In [11]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=gsm8k_devset, metric=gsm8k_metric, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

Average Metric: 6 / 10  (60.0): 100%|██████████| 10/10 [01:17<00:00,  7.79s/it]


60.0